In [14]:
#import the necessary packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import pickle
import joblib

In [2]:
# Load the dataset
data = pd.read_csv("final_merged_data.csv")

# Handle missing values (drop rows with NaN in lagged features)
data.dropna(inplace=True)


In [3]:
data.shape

(298946, 78)

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [5]:
data.head()

,last_reported,station_id,num_bikes_available,num_docks_available,is_installed,is_renting,is_returning,name,address,lat,lon,capacity,stno,year,month,day,hour,minute,max_air_temp_quality_indicator,max_air_temperature_celsius,min_air_temp_quality_indicator,min_air_temperature_celsius,air_temp_std_quality_indicator,air_temperature_std_deviation,max_grass_temp_quality_indicator,max_grass_temperature_celsius,min_grass_temp_quality_indicator,min_grass_temperature_celsius,grass_temp_std_quality_indicator,grass_temperature_std_deviation,max_soil_temp_5cm_quality_indicator,max_soil_temperature_5cm_celsius,min_soil_temp_5cm_quality_indicator,min_soil_temperature_5cm_celsius,soil_temp_std_5cm_quality_indicator,soil_temperature_std_deviation_5cm,max_soil_temp_10cm_quality_indicator,max_soil_temperature_10cm_celsius,min_soil_temp_10cm_quality_indicator,min_soil_temperature_10cm_celsius,soil_temp_std_10cm_quality_indicator,soil_temperature_std_deviation_10cm,max_soil_temp_20cm_quality_indicator,max_soil_temperature_20cm_celsius,min_soil_temp_20cm_quality_indicator,min_soil_temperature_20cm_celsius,soil_temp_std_20cm_quality_indicator,soil_temperature_std_deviation_20cm,max_earth_temp_30cm_quality_indicator,max_earth_temperature_30cm_celsius,min_earth_temp_30cm_quality_indicator,min_earth_temperature_30cm_celsius,earth_temp_std_30cm_quality_indicator,earth_temperature_std_deviation_30cm,max_earth_temp_50cm_quality_indicator,max_earth_temperature_50cm_celsius,min_earth_temp_50cm_quality_indicator,min_earth_temperature_50cm_celsius,earth_temp_std_50cm_quality_indicator,earth_temperature_std_deviation_50cm,max_earth_temp_100cm_quality_indicator,max_earth_temperature_100cm_celsius,min_earth_temp_100cm_quality_indicator,min_earth_temperature_100cm_celsius,earth_temp_std_100cm_quality_indicator,earth_temperature_std_deviation_100cm,max_humidity_quality_indicator,max_relative_humidity_percent,min_humidity_quality_indicator,min_relative_humidity_percent,humidity_std_quality_indicator,relative_humidity_std_deviation,max_pressure_quality_indicator,max_barometric_pressure_hpa,min_pressure_quality_indicator,min_barometric_pressure_hpa,pressure_std_quality_indicator,barometric_pressure_std_deviation
0,2024-12-01 00:10:00,10,15,1,True,True,True,DAME STREET,Dame Street,53.344006,-6.266802,16,175,2024,12,1,0,10,0,14.01,0,13.9,0,0.033,0,11.93,0,11.73,0,0.056,0,10.15,0,10.11,0,0.008,0,9.58,0,9.56,0,0.005,0,8.83,0,8.8,0,0.007,0,8.88,0,8.85,0,0.008,0,8.5,0,8.47,0,0.008,0,9.87,0,9.83,0,0.008,0,84.3,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
1,2024-12-01 00:10:00,100,17,8,True,True,True,HEUSTON BRIDGE (SOUTH),Heuston Bridge (South),53.347107,-6.292041,25,175,2024,12,1,0,10,0,14.01,0,13.9,0,0.033,0,11.93,0,11.73,0,0.056,0,10.15,0,10.11,0,0.008,0,9.58,0,9.56,0,0.005,0,8.83,0,8.8,0,0.007,0,8.88,0,8.85,0,0.008,0,8.5,0,8.47,0,0.008,0,9.87,0,9.83,0,0.008,0,84.3,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
2,2024-12-01 00:10:00,109,20,9,True,True,True,BUCKINGHAM STREET LOWER,Buckingham Street Lower,53.353333,-6.249319,29,175,2024,12,1,0,10,0,14.01,0,13.9,0,0.033,0,11.93,0,11.73,0,0.056,0,10.15,0,10.11,0,0.008,0,9.58,0,9.56,0,0.005,0,8.83,0,8.8,0,0.007,0,8.88,0,8.85,0,0.008,0,8.5,0,8.47,0,0.008,0,9.87,0,9.83,0,0.008,0,84.3,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
3,2024-12-01 00:10:00,11,1,29,True,True,True,EARLSFORT TERRACE,Earlsfort Terrace,53.334293,-6.258503,30,175,2024,12,1,0,10,0,14.01,0,13.9,0,0.033,0,11.93,0,11.73,0,0.056,0,10.15,0,10.11,0,0.008,0,9.58,0,9.56,0,0.005,0,8.83,0,8.8,0,0.007,0,8.88,0,8.85,0,0.008,0,8.5,0,8.47,0,0.008,0,9.87,0,9.83,0,0.008,0,84.3,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083
4,2024-12-01 00:10:00,114,4,36,True,True,True,WILTON TERRACE (PARK),Wilton Terrace (Park),53.333652,-6.248345,40,175,2024,12,1,0,10,0,14.01,0,13.9,0,0.033,0,11.93,0,11.73,0,0.056,0,10.15,0,10.11,0,0.008,0,9.58,0,9.56,0,0.005,0,8.83,0,8.8,0,0.007,0,8.88,0,8.85,0,0.008,0,8.5,0,8.47,0,0.008,0,9.87,0,9.83,0,0.008,0,84.3,0,83.2,0,0.284,0,1002.56,0,1002.26,0,0.083


In [6]:
# 添加平均温度与湿度（之前已加）
data['average_temperature'] = (data['max_air_temperature_celsius'] + data['min_air_temperature_celsius']) / 2
data['humidity'] = (data['max_relative_humidity_percent'] + data['min_relative_humidity_percent']) / 2

# 替代 day_of_the_week（1=Monday, 7=Sunday）
data['day_of_the_week'] = ((data['day'] + 6) % 7).replace(0, 7)

# ✅ 构造 timestamp 时间戳
data['timestamp'] = pd.to_datetime(data[['year', 'month', 'day', 'hour', 'minute']])

# ✅ 新特征：是否周末
data['is_weekend'] = data['day_of_the_week'].isin([6, 7]).astype(int)

# ✅ 新特征：是否上班高峰（早上7-9点 & 下午5-7点）
data['is_rush_hour'] = data['hour'].isin([7, 8, 17, 18]).astype(int)

In [7]:
# Define features and target
features = [
    'station_id', 'day_of_the_week', 'hour',
    'humidity', 'average_temperature',
    'is_weekend', 'is_rush_hour'
]

target = 'num_bikes_available'

X = data[features]
y = data[target]

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Ramdon Fores regression model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [9]:
# Make predictions
y_pred = model.predict(X_test)

In [10]:
print("Min predicted bikes:", y_pred.min())
print("Max predicted bikes:", y_pred.max())


Min predicted bikes: 0.0
Max predicted bikes: 40.0


Evaluate the model

In [11]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Absolute Error: {mae:.2f}")
print(f"R² Score: {r2:.4f}")

Mean Absolute Error: 1.47
R² Score: 0.9243


In [12]:
# 特征重要性
importances = model.feature_importances_
print("\nFeature Importances:")
for feature, importance in zip(features, importances):
    print(f"{feature}: {importance:.4f}")


Feature Importances:
station_id: 0.4046
day_of_the_week: 0.1269
hour: 0.1300
humidity: 0.1289
average_temperature: 0.1828
is_weekend: 0.0202
is_rush_hour: 0.0066


Saving the Model

In [15]:
# Save the model to a file
model_filename = "bike_availability_model_v1.joblib"
joblib.dump(model, model_filename)

print(f"Model saved to {model_filename}")

Model saved to bike_availability_model_v1.joblib


In [16]:
# Save the model to a .pkl file
model_filename = "bike_availability_model_v1.pkl"
with open(model_filename, "wb") as f:
    pickle.dump(model, f)

print(f"Model saved to {model_filename}")

Model saved to bike_availability_model_v1.pkl
